In [1]:
import gym
from ctypes import *
import numpy as np
import gym
from gym import error
from gym.utils import closer
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
from gym import spaces
from pypylon import pylon
import stable_baselines
import time
from PIL import Image
from pypylon import pylon
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



C:\Users\CIG\Anaconda3\envs\20oct_sb\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

In [3]:
class MyEnv(gym.Env):
    def __init__(self, camera,
                width = 600,
                height = 600):
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        
        self.camera = camera
        self.width = width
        self.height = height
        
        self.camera.Width.Value = self.width
        self.camera.Height.Value = self.height
        
        self.camera.BalanceWhiteAuto.SetValue("Off")
        self.camera.ExposureAuto.SetValue("Continuous")
        self.camera.GainAuto.SetValue("Off")
        
#         # get width and height from camera for making obs space
#         width = self.camera.Width.GetValue()
#         height = self.camera.Height.GetValue()
#         print(width, height)

#         # https://docs.baslerweb.com/features
#         self.camera.GainRaw.Value = GainRaw
#         self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
#         self.camera.AcquisitionFrameRateAbs.Value = AcquisitionFrameRateAbs
        
        
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        #focus
        x = c_double(54)
        self.lib.Casp_SetFocusVoltage(x)
        
        
        # 
        self.action_space = spaces.MultiDiscrete([3, #red channel: 0 -dont move; 1 - plus one, 2 - minus one
                                                  3, #green channel: 0 -dont move; 1 - plus one, 2 - minus one
                                                  3, #blue channel: 0 -dont move; 1 - plus one, 2 - minus one
                                                  #6, #exposure time: 0 -dont move; 1/2/3 - plus 1k/10k/100k, 
                                                    #4/5/6 - minus 1k/10k/100k
                                                  3, #gain: 0 -dont move; 1 - plus one, 2 - minus one
                                                  2, #0 - continue, 1 - stop
                                                 ])
        
        # image
        self.observation_space = spaces.Box(low=0, high=255, 
                                             shape=(self.height, self.width, 3), 
                                             dtype=np.float16)
        
        
        self.reset_path = r'D:\new_cringe_20oct\reset_image'
        self.stop_image = r'D:\new_cringe_20oct\stop_image'
        self.path = r'D:\new_cringe_20oct'
        self.i = 0
         
    def reset(self):
        print('reset')
        
        self.camera.BalanceRatioSelector.SetValue("Red")
        self.camera.BalanceRatio.SetValue(1)
        self.camera.BalanceRatioSelector.SetValue("Green")
        self.camera.BalanceRatio.SetValue(1)
        self.camera.BalanceRatioSelector.SetValue("Blue")
        self.camera.BalanceRatio.SetValue(1)
        
        self.camera.Gain.SetValue(0)
#         self.camera.ExposureTime.SetValue(20000)
        
        obs = self.grab_image()
        
        im = Image.fromarray(obs)
        filename = self.reset_path + "\img_%d.png" % (self.i)
        im.save(filename)
        
        return obs
    
    def step(self, action):
    
        print('action:', action)
        
        red_channel_action = action[0]
        green_channel_action = action[1]
        blue_channel_action = action[2]
#         exposure_time_action = action[3]
        gain_action = action[3]
        stop_action = action[4]
        
        reward = -0.1
        done = False
        
        if red_channel_action == 1:
            self.camera.BalanceRatioSelector.SetValue("Red")
            red_channel = self.camera.BalanceRatio.GetValue()
            try:
                self.camera.BalanceRatio.SetValue(int(red_channel) + 1)
            except:
                reward = -1
                done = True
        elif red_channel_action == 2:
            self.camera.BalanceRatioSelector.SetValue("Red")
            red_channel = self.camera.BalanceRatio.GetValue()
            try:
                self.camera.BalanceRatio.SetValue(int(red_channel) - 1)
            except:
                reward = -1
                done = True
            
        if green_channel_action == 1:
            self.camera.BalanceRatioSelector.SetValue("Green")
            green_channel = self.camera.BalanceRatio.GetValue()
            try:
                self.camera.BalanceRatio.SetValue(int(green_channel) + 1)
            except:
                reward = -1
                done = True
        elif green_channel_action == 2:
            self.camera.BalanceRatioSelector.SetValue("Green")
            green_channel = self.camera.BalanceRatio.GetValue()
            try:
                self.camera.BalanceRatio.SetValue(int(green_channel) - 1)
            except:
                reward = -1
                done = True
            
        if blue_channel_action == 1:
            self.camera.BalanceRatioSelector.SetValue("Blue")
            blue_channel = self.camera.BalanceRatio.GetValue()
            try:
                self.camera.BalanceRatio.SetValue(int(blue_channel) + 1)
            except:
                reward = -1
                done = True
        elif blue_channel_action == 2:
            self.camera.BalanceRatioSelector.SetValue("Blue")
            blue_channel = self.camera.BalanceRatio.GetValue()
            try:
                self.camera.BalanceRatio.SetValue(int(blue_channel) - 1)
            except:
                reward = -1
                done = True
        
#         exposure_time = self.camera.ExposureTime.GetValue()
#         if exposure_time_action == 1:
#             try:
#                 self.camera.ExposureTime.SetValue(exposure_time + 1000)
#             except:
#                 reward = -1
#                 done = True
#         elif exposure_time_action == 2:
#             try:
#                 self.camera.ExposureTime.SetValue(exposure_time + 10000)
#             except:
#                 reward = -1
#                 done = True
#         elif exposure_time_action == 3:
#             try:
#                 self.camera.ExposureTime.SetValue(exposure_time + 100000)
#             except:
#                 reward = -1
#                 done = True
#         elif exposure_time_action == 4:
#             try:
#                 self.camera.ExposureTime.SetValue(exposure_time - 1000)
#             except:
#                 reward = -1
#                 done = True
#         elif exposure_time_action == 5:
#             try:
#                 self.camera.ExposureTime.SetValue(exposure_time - 10000)
#             except:
#                 reward = -1
#                 done = True
#         elif exposure_time_action == 6:
#             try:
#                 self.camera.ExposureTime.SetValue( exposure_time - 100000)
#             except:
#                 reward = -1
#                 done = True
        
        gain = self.camera.Gain.GetValue()
        if gain_action == 1:
            try:
                self.camera.Gain.SetValue(int(gain) + 1)
            except:
                reward = -1
                done = True
        elif gain_action == 2:
            try:
                self.camera.Gain.SetValue(int(gain) - 1)
            except:
                reward = -1
                done = True
        
        
        obs = self.grab_image()
        
        if stop_action == 1:
            done = True
            ref_image = self.reference_image()
            reward = ssim(img_as_float(ref_image), img_as_float(obs), multichannel=True, data_range=ref_image.max() - ref_image.min())
            
            im = Image.fromarray(ref_image)
            filename = self.stop_image + "\img_%d.png" % (self.i)
            im.save(filename)
        
        im = Image.fromarray(obs)
        filename = self.path + "\img_%d.png" % (self.i)
        im.save(filename)
        
        self.i += 1
        
        return obs, reward, done, {}
    
    def reference_image(self):
        self.camera.BalanceWhiteAuto.SetValue("Continuous")
#         time.sleep(5)
#         self.camera.BalanceRatioSelector.SetValue(BalanceRatioSelector_Red)
#         red_channel = self.camera.BalanceRatioAbs.GetValue()
#         self.camera.BalanceRatioSelector.SetValue(BalanceRatioSelector_Green)
#         green_channel = self.camera.BalanceRatioAbs.GetValue()
#         self.camera.BalanceRatioSelector.SetValue(BalanceRatioSelector_Blue)
#         blue_channel = self.camera.BalanceRatioAbs.GetValue()
        
        #self.camera.ExposureAuto.SetValue("Continuous")
#         time.sleep(5)
#         exposure_time = self.camera.ExposureTimeRaw.GetValue()

        self.camera.GainAuto.SetValue("Continuous")
        n=0
        while n<15:
            reference_image = self.grab_image()
            n+=1
        time.sleep(10)
        reference_image = self.grab_image()
        
        self.camera.BalanceWhiteAuto.SetValue("Off")
        #self.camera.ExposureAuto.SetValue("Off")
        self.camera.GainAuto.SetValue("Off")
        time.sleep(5)
        
        return reference_image
        
        
                
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                #print(img[0])
                #print('shape:', img.shape)
                #plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image

In [4]:
env = MyEnv(camera)

eCOMCaspErr: 0 0


In [5]:
model = PPO2(MlpPolicy, env, tensorboard_log = r"D:\new_cringe_20oct\tf")





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [ ]:
model.learn(total_timesteps=10000)


reset
action: [2 1 2 1 0]
action: [2 1 1 0 0]
reset
action: [1 0 2 1 1]
reset
action: [1 0 1 0 0]
action: [1 1 1 1 1]
reset
action: [1 2 0 1 1]
reset
action: [1 1 0 1 0]
action: [0 2 0 1 0]
action: [1 0 1 2 1]
reset
action: [1 0 1 1 1]
reset
action: [1 1 1 1 0]
action: [2 0 0 0 0]
action: [1 1 2 0 0]
action: [2 2 0 0 1]
reset
action: [2 2 1 0 0]
action: [0 2 1 0 0]
reset
action: [0 0 2 2 1]
reset
action: [1 0 2 0 1]
reset
action: [0 0 1 1 1]
reset
action: [2 0 0 2 1]
reset
action: [2 2 1 1 1]
reset
action: [0 2 2 1 1]
reset
action: [1 0 0 2 0]
reset
action: [2 1 0 0 0]
action: [1 1 2 0 0]
action: [0 0 2 2 1]
reset
action: [0 0 2 1 0]
action: [1 0 0 2 0]
reset
action: [2 1 2 2 0]
reset
action: [1 1 1 1 0]
action: [0 0 2 1 0]
action: [0 2 2 2 0]
reset
action: [2 0 2 2 0]
reset
action: [1 0 1 0 0]
action: [1 1 0 2 1]
reset
action: [0 0 2 1 0]
action: [0 2 2 2 1]
reset
action: [2 1 0 2 0]
reset
action: [0 1 2 1 1]
reset
action: [2 0 1 2 0]
reset
action: [1 1 0 1 1]
reset
action: [1 0 2 0 

C:\Users\CIG\Anaconda3\envs\20oct_sb\lib\site-packages\skimage\metrics\_structural_similarity.py:208: RuntimeWarning: invalid value encountered in true_divide
  S = (A1 * A2) / D


reset
action: [2 0 1 2 1]
reset
action: [2 0 1 1 0]
action: [2 0 2 0 0]
reset
action: [0 2 2 0 1]
reset
action: [1 1 2 1 1]
